In [1]:
import os

import numpy as np
import pandas as pd

#from pandas_profiling import ProfileReport

In [2]:
INPUT_FOLDER = "enrich"
OUTPUT_FOLDER = ""

In [4]:
key = "us_shareproce_joined_companies"

share_prices = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.csv")
share_prices.info(show_counts=True)
share_prices['Date'] = pd.to_datetime(share_prices['Date'])

share_prices.set_index('Date', inplace=True)  # Set Date as the index

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5380108 entries, 0 to 5380107
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Date          5380108 non-null  object 
 1   Industry      5380108 non-null  object 
 2   Sector        5380108 non-null  object 
 3   Ticker        5380108 non-null  object 
 4   Open          5380108 non-null  float64
 5   High          5380108 non-null  float64
 6   Low           5380108 non-null  float64
 7   Close         5380108 non-null  float64
 8   Volume        5380108 non-null  int64  
 9   Dividend      5380108 non-null  float64
 10  Company Name  5380108 non-null  object 
dtypes: float64(5), int64(1), object(5)
memory usage: 451.5+ MB


In [5]:
share_prices.isna().sum()

Industry        0
Sector          0
Ticker          0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividend        0
Company Name    0
dtype: int64

In [7]:
df = share_prices[share_prices['Ticker'] == 'A']
df.head()

,Industry,Sector,Ticker,Open,High,Low,Close,Volume,Dividend,Company Name
Date,,,,,,,,,,
2019-04-11,Medical Diagnostics & Research,Healthcare,A,81.88,81.92,80.89,81.08,1071479,0.0,AGILENT TECHNOLOGIES INC
2019-04-12,Medical Diagnostics & Research,Healthcare,A,81.43,82.06,80.90,80.98,1249295,0.0,AGILENT TECHNOLOGIES INC
2019-04-15,Medical Diagnostics & Research,Healthcare,A,81.00,81.13,79.91,80.40,1627268,0.0,AGILENT TECHNOLOGIES INC
2019-04-16,Medical Diagnostics & Research,Healthcare,A,80.82,80.96,77.19,77.55,3441597,0.0,AGILENT TECHNOLOGIES INC
2019-04-17,Medical Diagnostics & Research,Healthcare,A,78.15,78.32,74.46,75.43,4471971,0.0,AGILENT TECHNOLOGIES INC


Clasification model #1 (0 = down, 1 = up)

In [ ]:
df['Change'] = df['Close'].diff().apply(lambda x: 1 if x > 0 else 0)
df.head()

/var/folders/nl/1mz19h4n1d920hbz6_952lk00000gn/T/ipykernel_45608/3227524502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Change'] = df['Close'].diff().apply(lambda x: 1 if x > 0 else 0)


,Industry,Sector,Ticker,Open,High,Low,Close,Volume,Dividend,Company Name,Change
Date,,,,,,,,,,,
2019-04-11,Medical Diagnostics & Research,Healthcare,A,81.88,81.92,80.89,81.08,1071479,0.0,AGILENT TECHNOLOGIES INC,0
2019-04-12,Medical Diagnostics & Research,Healthcare,A,81.43,82.06,80.90,80.98,1249295,0.0,AGILENT TECHNOLOGIES INC,0
2019-04-15,Medical Diagnostics & Research,Healthcare,A,81.00,81.13,79.91,80.40,1627268,0.0,AGILENT TECHNOLOGIES INC,0
2019-04-16,Medical Diagnostics & Research,Healthcare,A,80.82,80.96,77.19,77.55,3441597,0.0,AGILENT TECHNOLOGIES INC,0
2019-04-17,Medical Diagnostics & Research,Healthcare,A,78.15,78.32,74.46,75.43,4471971,0.0,AGILENT TECHNOLOGIES INC,0


In [ ]:
df.corr()

In [ ]:
from collections import Counter

def class_perc(data):
    lendata = len(data)
    classes = Counter(data)

    for sclass, freq in classes.items():
        perc = (freq / lendata) * 100
        print(f"Class '{sclass}': {perc:.2f}%", ", freq:",freq)

class_perc(df['Change'])

In [ ]:
from sklearn.model_selection import train_test_split

def split_df(dataframe, seed=None, percentage=0.8):

    X = df.loc[:, dataframe.columns != 'Change']
    y = df['Change']

    return train_test_split(X, y, test_size=1-percentage, random_state=seed, stratify=y) # note the stratify parameter

X_train, X_test, Y_train, Y_test = split_df(df, seed=42, percentage=0.5)

In [ ]:
print("Training data set: ", X_train.shape)
print("Testing data set: ", X_test.shape)
class_perc(Y_train.to_frame(name='Change')["Change"])
class_perc(Y_test.to_frame(name='Change')["Change"])

In [ ]:
from sklearn.tree import DecisionTreeClassifier

shares_tree = DecisionTreeClassifier(criterion="entropy", random_state=42)
shares_tree.fit(X_train, Y_train)

In [ ]:
from sklearn.metrics import auc, roc_curve, f1_score, accuracy_score, precision_recall_curve, confusion_matrix, classification_report
from sklearn.metrics import recall_score, precision_score
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score

pred_train = shares_tree.predict(X_train)
print("Accuracy of training set = {0:.2%}".format(accuracy_score(Y_train, pred_train)))

pred_test = shares_tree.predict(X_test)
print("Accuracy of testing set = {0:.2%}".format(accuracy_score(Y_test, pred_test)))

#Very high overfitting

In [ ]:
from io import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

def plot_tree(tree, feature_names):
    dot_data = StringIO()
    export_graphviz(tree, out_file=dot_data, feature_names=feature_names,
                    filled=True, rounded=True,special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    graph.write_png("titanic_train.png")
    return Image(graph.create_png())

plot_tree(shares_tree, X_train.columns)

In [ ]:
root_node=shares_tree.tree_
node=0
print("Samples: ", root_node.n_node_samples[node])
print("Proportions per class: ",root_node.value[node])
print("Samples per class: ", root_node.value[node]*root_node.n_node_samples[node])
print("Impurity: {0:.2%}".format(root_node.impurity[node]))

In [ ]:
print("Number of nodes: ", shares_tree.tree_.node_count)
print("Number of leaves: ", shares_tree.get_n_leaves())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
importances = shares_tree.feature_importances_
indices = np.argsort(importances)[::-1]
plt.bar(X_train.columns[indices], importances[indices]) #average reduction in impurity resulting from splitting at each node of the tree using that feature
plt.title('Feature Importance', fontsize=16);

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': range(1,9)} # 9 different depth levels

shares_tree_pruned_cv = GridSearchCV(shares_tree,
                   param_grid,
                   scoring='accuracy',
                   cv=5 , n_jobs=1, verbose=1)

shares_tree_pruned_cv.fit(X_train,Y_train);

In [ ]:
print("Best parameters set found on development set:")
print()
print(shares_tree_pruned_cv.best_params_)
print()
print("Grid scores on development set:")
print()
means = shares_tree_pruned_cv.cv_results_['mean_test_score']
stds = shares_tree_pruned_cv.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, shares_tree_pruned_cv.cv_results_['params']):
    print("Accuracy = %0.3f (+/%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
plt.figure(figsize=(10,8))
plt.errorbar(range(1,9,1), [m for m in means], yerr=stds, fmt='--o')
plt.title('Accuracy for different Depths', fontsize=20)
plt.xlabel("Depth", fontsize=16)
plt.ylabel("Accuracy", fontsize=16);

In [ ]:
shares_tree_pruned = DecisionTreeClassifier(random_state=42, max_depth=5, class_weight='balanced')
tree = shares_tree_pruned.fit(X_train, Y_train)

probabilities = shares_tree_pruned.predict_proba(X_test)[:, 1]

threshold = 0.5
predictions_test = (probabilities >= threshold).astype(int)

print("Accuracy = {0:.2%}".format(accuracy_score(Y_test, predictions_test)))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

prob_pred = tree.predict_proba(X_test)[:, 1]
thresholds = np.arange(0.0, 1.0, step=0.1)
recall_scores = [metrics.recall_score(Y_test, prob_pred > t, average='macro') for t in thresholds]
precis_scores = [metrics.precision_score(Y_test, prob_pred > t, average='macro') for t in thresholds]

fig, ax = plt.subplots(1, 1)
ax.plot(thresholds, recall_scores, label="Recall @ t")
ax.plot(thresholds, precis_scores, label="Precision @ t")
ax.axvline(0.5, c="gray", linestyle="--", label="Default Threshold")
ax.set_xlabel("Threshold")
ax.set_ylabel("Metric @ Threshold")
ax.set_box_aspect(1)
ax.legend()
plt.show()

In [ ]:
print("Number of nodes: ", shares_tree_pruned.tree_.node_count)
print("Number of leaves: ", shares_tree_pruned.get_n_leaves())

In [ ]:
from sklearn import tree

text_representation = tree.export_text(shares_tree_pruned, feature_names=list(X_test.columns))
print(text_representation)

# optionally, we can export them to a text file
with open("shares_pruned.log", "w") as fout:
    fout.write(text_representation)

In [ ]:
import graphviz
# DOT data
dot_data = tree.export_graphviz(shares_tree_pruned, out_file=None,
                                class_names=["No","Yes"],
                                feature_names=X_train.columns,
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png")
graph

Bagging (1 up, -1 down)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,df.columns != 'Change'],
                                                    df['Change'],
                                                    test_size=0.25,
                                                    random_state=42)

In [ ]:
print("Train: ", x_train.shape)
print("Test: ", x_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

baseRF = RandomForestClassifier(n_estimators = 200,  # The number of trees in the forest
                               random_state = 42, # This parameter allows to replicate results
                               n_jobs = -1, # The number of jobs to run in parallel (-1 means all processors, 1 no parallelism)
                               oob_score = True,
                               class_weight='balanced') # Whether to use out-of-bag samples to estimate the generalization score (increases time)

# The OOB score is an estimate of the generalization error of the model, calculated with the data not used to train it.
# Since each tree is trained on a bootstrapped sample, they all use approximately 2/3 of the data (this is a mathematical consequence
# of random sampling with replacement). Therefore, for each of the learners we have 1/3 of the dataset as Out Of Bag (OOB) observations
# which can be used to test the model and estimate its generalisation error. By averaging these OOB error estimates across the trees,
# RF produces a robust error estimate for the ensemble during training time at almost no additional computational cost.

# other parameters:
# criterion{"gini", "entropy", "log_loss"}, default="gini"
# max_depth: int, default=None
# min_samples_leaf: int or float, default=1 - The minimum number of samples required to be at a leaf node
# min_samples_split: int or float, default=2 - The minimum number of samples required to split an internal node:
# max_features{"sqrt", "log2", None}, int or float, default="sqrt" - The number of features to consider when looking for the best split

baseRF.fit(x_train,y_train)

# predictions
y_pred_test = baseRF.predict(x_test)

print("OOB score: {:.2%}".format(baseRF.oob_score_))

In [ ]:
param_grid = {
    'min_samples_leaf' :[10,15,20],
    'max_features' : [4,6,8],
    'criterion' : ['gini','entropy'] #,'log_loss']
}

In [ ]:
import time
start_time = time.time()

from sklearn.model_selection import GridSearchCV
cvRF = GridSearchCV(estimator=baseRF, param_grid=param_grid, cv=5, scoring='roc_auc')
#cv is the number of cross validation iterations to be performed
cvRF.fit(x_train,y_train)

print ("Completed in {:0.0f} seconds ".format((time.time() - start_time)))

In [ ]:
best_params = cvRF.best_params_
print("Best parameters:", best_params)

In [ ]:
print("ROC AUC: ", '{0:0.2%}'.format(cvRF.best_score_))

In [ ]:
import time
start_time = time.time()

from sklearn.model_selection import GridSearchCV
cvRF = GridSearchCV(estimator=baseRF, param_grid=param_grid, cv=5, scoring='roc_auc')
#cv is the number of cross validation iterations to be performed
cvRF.fit(x_test,y_test)

In [ ]:
print("ROC AUC: ", '{0:0.2%}'.format(cvRF.best_score_))

#Again showing very high overfitting.

In [ ]:
y_pred = cvRF.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Random Forest

In [ ]:
model_RF = RandomForestClassifier(n_estimators = 200,
                               random_state = 0,
                               max_features = 6, #this parameter makes the difference between simple Bagging and Random Forests
                               n_jobs = -1,
                               oob_score = True,
                               # criterion = 'entropy',
                               min_samples_leaf = 10)
model_RF.fit(x_train,y_train)

# predictions
y_pred_test = model_RF.predict(x_test)

In [ ]:
print("OOB score: {:.2%}".format(model_RF.oob_score_))

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, precision_recall_curve, confusion_matrix, recall_score, precision_score

pred_prob = model_RF.predict_proba(x_test) 
prob_malign = [p[1] for p in pred_prob] 
auc = roc_auc_score(y_test, prob_malign)
print("ROC AUC score: {:.2%}".format(auc))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, prob_malign, pos_label=1)

train_acc = round(model_RF.score(x_train,y_train) * 100,2) 
test_acc = round(model_RF.score(x_test,y_test) * 100,2) 
print("Train Accuracy score: ", train_acc, "%")
print("Test Accuracy score: ", test_acc, "%")

In [ ]:
importances = model_RF.feature_importances_

indexes = np.argsort(importances)[::-1]
sorted_imp = importances[indexes]

plt.figure(figsize=(12, 6))
bars = plt.bar(range(len(importances)), importances[indexes], align='center')
plt.xticks(range(len(importances)), np.array(model_RF.feature_names_in_)[indexes], rotation=90)
plt.xlabel("Features")
plt.ylabel("Importance")
plt.xticks(rotation=90, fontsize=8)
plt.title("Relative feature importances (from 0 to 1)")
for i, bar in enumerate(bars):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{sorted_imp[i]:.2f}', ha='center', va='bottom')
plt.show()

#Dividend is not important at all for the analysis - can be deleted

In [ ]:
train_scores = []
cv_scores    = []

estimator_range = range(100, 300, 25) #from 5 trees to 50, step 5

for n_estimators in estimator_range:
    model_RF = RandomForestClassifier(
                n_estimators = n_estimators,
                max_features = 5, # the previous optimum
                oob_score    = False,
                n_jobs       = -1,
                random_state = 100,
                class_weight='balanced'
             )

    model_RF.fit(x_train, y_train)
    predictions = model_RF.predict(x_test)
    acc=accuracy_score(y_test, predictions)
    print("% Accuracy of test dataset for {} trees is {:.3f}".format(n_estimators, acc))
    train_scores.append(acc) # storing accuracy from each iteration

    # and we also store the accuracy mean obtained from running a 5-fold validation
    cvscores = cross_val_score(
                estimator = model_RF,
                X         = x_train,
                y         = y_train,
                scoring   = 'accuracy',
                cv        = 5
             )
    cv_scores.append(cvscores.mean())

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(estimator_range, train_scores, marker='o', linestyle='-', color='b')
plt.title('Accuracy of Test Dataset vs. Number of Trees')
plt.xlabel('Number of Trees (n_estimators)')
plt.ylabel('Accuracy')
plt.xticks(estimator_range)  # Para mostrar todos los valores en el eje x
plt.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(estimator_range, train_scores, label="train scores")
ax.plot(estimator_range, cv_scores, label="cv scores")
ax.plot(estimator_range[np.argmax(cv_scores)], max(cv_scores),
        marker='o', color = "red", label="max score")
ax.set_ylim(0.65, 0.70)
ax.set_ylabel("% Accuracy")
ax.set_xlabel("n_estimators")
ax.set_title("Evolution of cv-error vs. number of trees")
plt.legend();
print(f"Optimum number of trees: {estimator_range[np.argmax(cv_scores)]}")

In [ ]:
model_RF = RandomForestClassifier(n_estimators = 200,
                               random_state = 100,
                               max_features = 6, #this parameter makes the difference between simple Bagging and Random Forests
                               n_jobs = -1,
                               oob_score = True,
                               # criterion = 'entropy',
                               min_samples_leaf = 10,
                               class_weight='balanced')
model_RF.fit(x_train,y_train)

# predictions
y_pred_test = model_RF.predict(x_test)

In [ ]:
print("OOB score: {:.2%}".format(model_RF.oob_score_))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, prob_malign, pos_label=1)

train_acc = round(model_RF.score(x_train,y_train) * 100,2) #Train Accuracy score
test_acc = round(model_RF.score(x_test,y_test) * 100,2) #Test Accuracy score
print("Train Accuracy score: ", train_acc, "%")
print("Test Accuracy score: ", test_acc, "%")

Regression

In [ ]:
import pandas as pd, numpy as np             # Data manipulation
from matplotlib import pyplot as plt         # Graphing
import seaborn as sns                        # Graphing

import statsmodels.api as sm                    # Statistical analysis
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy import stats                      # Stats

#sns.set(style="white")                       # Tuning the style of charts
import warnings                              # Disable some warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
df.head()

In [ ]:
df_model, df_reserved = train_test_split(df, test_size=0.1, random_state=42)
print ("Sample size dataset reserved for prediction: ", df_reserved.shape[0], "records")

target = 'Change'

y = df_model[target]

# cross validation (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(df_model, y, test_size=0.2)
print ("Sample size train dataset: ", X_train.shape)
print ("Sample size test dataset: ", X_test.shape)

In [ ]:
# Stepwise function
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.05,
                       threshold_out = 0.1,
                       verbose=True):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features
    Always set threshold_in < threshold_out to avoid infinite looping.
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [ ]:
result = stepwise_selection(X_train, y_train)     #For logit regression, stepwise does an iterative process
print('resulting features:')
print(result) # relevant variables
type(result)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

sns.set(font_scale=1.1)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(X_train_stepwise.corr(), annot=True, fmt=".2f", linewidths=1, ax=ax)